Documentation de l'API : https://github.com/GeneralMills/pytrends

### open issues
Limites de requêtes sur Google :
- générer un string aléatoire à passer en paramètre custom_useragent
https://github.com/GeneralMills/pytrends/issues/77
- si erreur, utiliser un autre compte gmail

Format JSON :
- quel format lu par chart.js, d3.js ?
http://stackoverflow.com/questions/24929931/drawing-line-chart-in-chart-js-with-json-response

Plus de 5 recherches à la fois :
- cf open issues sur pytrends
https://github.com/GeneralMills/pytrends/issues/77

Gérer les timestamps pour les extracts de données sans heure ni minute

### Les tendances sur les recherches
- 'Élection présidentielle française de 2017' : '/m/0n49cn3' (Élection)

#### Codes associés aux candidats :
- Manuel Valls : '/m/047drb0' (Ancien Premier ministre français)
- François Fillon : '/m/0fqmlm' (Ancien Ministre de l’Écologie, du Développement durable et de l’Énergie)
- Vincent Peillon : '/m/0551vp' (Homme politique)
- François Bayrou : '/m/02y2cb' (Homme politique)
- François Hollande : '/m/02qg4z' (Président de la République française)
- Jean-Luc Mélanchon : '/m/04zzm99' (Homme politique)
- Yannick Jadot : '/m/05zztc0' (Ecologiste)
- Nicolas Dupont-Aignan (Debout la France) : '/m/0f6b18'
- Michèle Alliot-Marie (Indépendante) : '/m/061czc' (Ancienne Ministre de l'Intérieur de France)
- Nathalie Artaud (LO) : pas dispo
- Philippe Poutou (NPA) : '/m/0gxyxxy'
- Emmanuel Macron : '/m/011ncr8c' (Ancien Ministre de l'Économie et des Finances)
- Jacques Cheminade : '/m/047fzn'
- Marine Le Pen : '/m/02rdgs'

#### Codes associés aux partis :
- LR : '/g/11b7n_r2jq'
- PS : '/m/01qdcv'
- FN : '/m/0hp7g'
- EELV : '/m/0h7nzzw'
- FI (France Insoumise) : pas dispo direttement... utiliser le terme de recherche : "france insoumise"
- PCF : '/m/01v8x4'
- Debout la France : '/m/02rwc3q'
- MoDem : '/m/02qt5xp' (Mouvement Démocrate)
- Lutte Ouvrière : '/m/01vvcv'
- Nouveau Parti Anticapitalise : '/m/04glk_t'
- En marche! : pas dispo

#### Codes couleurs :

- dark blue: #637b85
- green: #2c9c69
- yellow: #dbba34
- red: #c62f29


In [24]:
#!/usr/bin/python
# coding: utf-8

from __future__ import unicode_literals
from trendsAPI import * # API non officielle
import json
import pandas as pd
from datetime import datetime, timedelta
import re
import sys
from numpy.random import rand
from numpy import sign
import time


def convert_date_column(dataframe): # Conversion du format en un string court
    dates = []
    rdict = {',': '', ' PST': '', ' à': '', 'janv.': '01', 'févr.': '02', 'mars': '03', 'avr.': '04',
             'mai': '05', 'juin': '06', 'juil.': '07', 'août': '08', 'sept.': '09', 'oct.': '10',
             'nov.': '11', 'déc.': '12'}
    robj = re.compile('|'.join(rdict.keys()))
    
    rdict2 = {' 01 ': ' janv. ', ' 02 ': ' févr. ', ' 03 ': ' mars ', ' 04 ': ' avr. ', ' 05 ': ' mai ',
              ' 06 ': ' juin ', ' 07 ': ' juil. ', ' 08 ': ' août ', ' 09 ': ' sept. ', ' 10 ': ' oct. ',
              ' 11 ': ' nov. ', ' 12 ': ' déc. '}
    robj2 = re.compile('|'.join(rdict2.keys()))
    
    if 'PST' in dataframe['Date'][0]: # format de date anglais avec heure
        in_format = '%b %d %Y %H:%M' # type : Jan 18 2017 12:00
        
        for date in dataframe['Date']: # Conversion en timestamp sur le fuseau GMT+1
            t = datetime.strptime(robj.sub(lambda m: rdict[m.group(0)], date), in_format) + timedelta(hours=9)
            t = datetime.strftime(t, '%d %m %H:%M') # conversion de nouveau en string du type : 18 01 12:00
            dates.append(robj2.sub(lambda m: rdict2[m.group(0)], t)) # remplacement des mois en toutes lettres

    elif 'UTC' in dataframe['Date'][0]: # format de date français avec heure
        in_format = '%d %m %Y %H:%M' # type : 18 01 2017 12:00
        
        for date in dataframe['Date']: # Conversion en timestamp sur le fuseau GMT+1
            t = datetime.strptime(robj.sub(lambda m: rdict[m.group(0)], date[0:-6]), in_format) + timedelta(hours=9)
            t = datetime.strftime(t, '%d %m %H:%M') # conversion de nouveau en string du type : 18 01 12:00
            dates.append(robj2.sub(lambda m: rdict2[m.group(0)], t)) # remplacement des mois en toutes lettres
    
    else: # si les dates ne contiennent pas l'heure (ie. recherche sur plus d'un mois)
        rdict = {', ': ' ', 'janvier': 'janv.', 'février': 'févr.', 'avril': 'avr.', 'juillet': 'juil.',
                 'septembre': 'sept.', 'octobre': 'oct.', 'novembre': 'nov.', 'décembre': 'déc.'}
        robj = re.compile('|'.join(rdict.keys()))
        for date in dataframe['Date']:
            t = robj.sub(lambda m: rdict[m.group(0)], date)
            dates.append(' '.join(t.split(' ')[1:-1]))
    
    dataframe['Date'] = dates
    return


def trends_to_json(query='', periode=''):
    """
    Télécharge sous format json les données de Google Trends avec les paramètres indiqués.
    Ceux-ci doivent appartenir aux recherches préconfigurées dans les dictionnaires <queries>
    et <periodes>.
    
    Si aucun paramètre n'est spécifié, la fonction va balayer toutes les combinaisons de
    requêtes et de périodes préconfigurées.
    """

    # Les termes de recherche (5 au maximum separes par des virgules)
    # On associe a un type de recherche la liste des parametres correspondants
    queries = {'candidats_A': '/m/047drb0, /m/04zzm99, /m/02rdgs, /m/011ncr8c, /m/0fqmlm',
               'partis_A': '/g/11b7n_r2jq, /m/01qdcv, /m/0hp7g, /m/0h7nzzw',
              'divers_gauche': 'France Insoumise, /m/01vvcv, /m/04glk_t, /m/01v8x4'} 
    
    periodes = {'3d': 'now 3-d'}

    if query == '':
        query = set([q for q in queries])
    else:
        query = set([query])
    
    if periode == '':
        periode = set([p for p in periodes])
    else:
        periode = set([periode])
        
    if not (query.issubset(queries) and periode.issubset(periodes)):
        print('Erreur de parametre')
        return
    
    success = []
    users = ['pfrlepoint@gmail.com', 'pfrlepoint2@gmail.com']
    for user in users[::int(sign(rand(1) * 2 - 1))]: # une chance sur deux de partir de la fin de la liste
        try:
            # Connection à Google (utiliser une vraie adresse gmail permet plus de requêtes)
            pytrend = TrendReq(user, 'projet_fil_rouge', custom_useragent='PFR')
            
            for q in query:
                for p in periode:
                    if (q, p) in success:
                        continue
                    else:
                        payload = {'q': queries[q], 'geo': 'FR', 'date': periodes[p], 'hl': 'fr-FR'}
                        # Possibilite de periode personnalise : specifier deux dates (ex : 2015-01-01 2015-12-31)
                        # geographie : FR (toute France), FR-A ou B ou C... (region de France par ordre alphabetique)
                        # categorie politique : cat = 396

                        df = pytrend.trend(payload, return_type='dataframe')
                        convert_date_column(df) # converts date into a short string
                        df.set_index('Date', inplace=True)

                        # reduction du nombre de lignes du dataframe a une trentaine de points
                        # pour la lisibilité du graph
                        n = {'4h': 2, '1d': 4, '3d': 1, '7d': 6, '1m': 1, '3m': 2}
                        # n = 1 # pour désactiver cette fonction

                        # Sauvegarde en JSON
                        server_path = '/var/www/html/gtrends/data/' # path complet
                        # server_path = ''
                        df[(df.shape[0] - 1) % n[p]::n[p]].to_json(
                            server_path + q + '_' + p + '.json', orient='split', date_unit='ms')

                        print('Connexion réussie avec l\'adresse : ' + user)
                        print('Enregistrement sous : ' + server_path + q + '_' + p + '.json')
                        success.append((q, p)) # on garde en mémoire les couples q, p qui ont fonctionné

                        # espacement des requêtes pour ne pas dépasser la limite
                        time.sleep(6)
                    
            return

        except (RateLimitError, ResponseError):
            print('Limite de requêtes dépassée, tentative avec une autre adresse mail...')

    print('Erreur lors de la récupération des données.')
    return


#### Fonction qui sauvegarde les requetes via l'API en JSON

In [25]:
trends_to_json()

Connexion réussie avec l'adresse : pfrlepoint2@gmail.com
Enregistrement sous : partis_A_3d.json
Connexion réussie avec l'adresse : pfrlepoint2@gmail.com
Enregistrement sous : candidats_A_3d.json
Connexion réussie avec l'adresse : pfrlepoint2@gmail.com
Enregistrement sous : divers_gauche_3d.json


In [23]:
df_json = pd.read_json('partis_A_3d.json', orient='split')
df_json

,Europe Ecology – The Greens,National Front,Socialist Party,The Republicans
20 janv. 12:00,0,5,8,1
20 janv. 13:00,0,5,8,1
20 janv. 14:00,0,4,5,1
20 janv. 15:00,0,4,5,1
20 janv. 16:00,0,4,5,1
20 janv. 17:00,0,3,5,1
20 janv. 18:00,0,4,6,1
20 janv. 19:00,0,5,8,1
20 janv. 20:00,0,5,10,1
20 janv. 21:00,0,4,6,1


In [16]:
payload = {'q': 'France Insoumise, /m/01vvcv, /m/04glk_t, /m/01v8x4', 'geo': 'FR',
           'date': 'now 4-H', 'hl': 'fr-FR'}
pytrend = TrendReq('pfrlepoint@gmail.com', 'projet_fil_rouge', custom_useragent='pfr')
df = pytrend.trend(payload, return_type='dataframe')
df

,Date,French Communist Party,Lutte Ouvrière,New Anticapitalist Party,france insoumise
0,22 janv. 2017 à 22:40 UTC−8,46,15,31,0
1,22 janv. 2017 à 22:48 UTC−8,87,72,29,0
2,22 janv. 2017 à 22:56 UTC−8,27,41,14,14
3,22 janv. 2017 à 23:04 UTC−8,25,50,12,12
4,22 janv. 2017 à 23:12 UTC−8,35,23,23,12
5,22 janv. 2017 à 23:20 UTC−8,23,11,11,10
6,22 janv. 2017 à 23:28 UTC−8,42,31,10,10
7,22 janv. 2017 à 23:36 UTC−8,58,39,10,10
8,22 janv. 2017 à 23:44 UTC−8,9,18,18,36
9,22 janv. 2017 à 23:52 UTC−8,43,17,15,9


In [33]:
df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)
dates = []
for elem in df['Date']:
    dates.append(datetime.strftime(elem, '%d/%m'))
df['Date'] = dates

ValueError: Unknown string format

In [85]:
periodes = {'4h': 'now 4-H', '1d': 'now 1-d', '3d': 'now 3-d', '7d': 'now 7-d', '1m': 'today 1-m'}
a = set([x for x in periodes])
for eleme, value in periodes:
    print(eleme, value)
print(not set('4h').issubset(periodes))

3 d
7 d
4 h
1 d
1 m
True


In [90]:
set(['1d']).issubset(set(periodes))

True

In [88]:
set(periodes)

{'1d', '1m', '3d', '4h', '7d'}

In [92]:
set(['1d'])

{'1d'}

In [96]:
for a, b in {1,2}, {2,3}:
    print(a,b)

1 2
2 3


In [98]:
a = [(1,2), (2,3)]
(1,2) in a

True